We read our dataframe.

In [38]:
import pandas as pd

df = pd.read_csv('../candidates.csv' , sep=';')

In [39]:
df.rename(columns={
        'First Name': 'first_name',
        'Last Name': 'last_name',
        'Email': 'email',
        'Application Date': 'application_date',
        'Country': 'country',
        'YOE': 'yoe',
        'Seniority': 'seniority',
        'Technology': 'technology',
        'Code Challenge Score': 'code_challenge_score',
        'Technical Interview Score': 'technical_interview'
    }, inplace=True)

print(df.columns)

Index(['first_name', 'last_name', 'email', 'application_date', 'country',
       'yoe', 'seniority', 'technology', 'code_challenge_score',
       'technical_interview'],
      dtype='object')


Connection to our database "workshop_01"

In [33]:
import psycopg2
import json

def create_connection():
    try:
        with open('../credentials.json') as f:
            credentials = json.load(f)
        
        user = credentials['user']
        password = credentials['password']
        host = credentials['host']
        port = credentials['port']
        database = 'workshop_01'
        
        connection = psycopg2.connect(
            dbname=database,
            user=user,
            password=password,
            host=host,
            port=port
        )
        print("¡Conexión exitosa!")
        return connection
    except psycopg2.OperationalError as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None

create_connection()

¡Conexión exitosa!


<connection object at 0x000001EED1C0F450; dsn: 'user=postgres password=xxx dbname=workshop_01 host=localhost port=5432', closed: 0>

We create our table "workshop_01"

In [34]:
def create_table():
    connection = create_connection()
    if connection is not None:
        try:
            cursor = connection.cursor()
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS workshop_01 (
                    first_name VARCHAR(150),
                    last_name VARCHAR(150),
                    email VARCHAR(100),
                    country VARCHAR(150),
                    application_date DATE,
                    yoe INTEGER,
                    seniority VARCHAR(150),
                    technology VARCHAR(100),
                    code_challenge_score DECIMAL(5, 2),
                    technical_interview VARCHAR(150)
                );
            """)
            connection.commit()
            print("Tabla 'workshop_01' creada con éxito en la base de datos.")
        except psycopg2.Error as e:
            print(f"Error al crear la tabla: {e}")
        finally:
            cursor.close()
            connection.close()
    else:
        print("No se pudo establecer la conexión con la base de datos.")

create_table()

¡Conexión exitosa!
Tabla 'workshop_01' creada con éxito en la base de datos.


Insert the data from our dataframe to our databases

In [40]:
def insertar_datos(df):
    connection = create_connection()
    if connection is not None:
        cursor = connection.cursor()
        query = """
        INSERT INTO workshop_01 (
            first_name, last_name, email, country, application_date,
            yoe, seniority, technology, code_challenge_score, technical_interview
        ) VALUES (
            %(first_name)s, %(last_name)s, %(email)s, %(country)s, %(application_date)s,
            %(yoe)s, %(seniority)s, %(technology)s, %(code_challenge_score)s, %(technical_interview)s
        )
        """
        try:
            for index, row in df.iterrows():
                data = row.to_dict()
                cursor.execute(query, data)
            connection.commit()
            print("Datos insertados exitosamente")
        except (Exception, psycopg2.DatabaseError) as error:
            print(f"Error al insertar datos: {error}")
        finally:
            cursor.close()
            connection.close()
    else:
        print("No se pudo establecer la conexión con la base de datos.")

# Insertar los datos en la base de datos
insertar_datos(df)


¡Conexión exitosa!
Datos insertados exitosamente


I show the first 5 results from our database

In [41]:
def mostrar_primeros_resultados(tabla, num_filas=3):
    connection = create_connection()
    if connection is not None:
        try:
            cursor = connection.cursor()
            query = f"SELECT * FROM {tabla} LIMIT %s;"
            cursor.execute(query, (num_filas,))
            resultados = cursor.fetchall()
            
            column_names = [desc[0] for desc in cursor.description]
            print(f"Resultados de la tabla '{tabla}':")
            print(f"{' | '.join(column_names)}")
            for fila in resultados:
                print(fila)
        except (Exception, psycopg2.DatabaseError) as error:
            print(f"Error al mostrar resultados: {error}")
        finally:
            cursor.close()
            connection.close()
    else:
        print("No se pudo establecer la conexión con la base de datos.")

mostrar_primeros_resultados('workshop_01')

¡Conexión exitosa!
Resultados de la tabla 'workshop_01':
first_name | last_name | email | country | application_date | yoe | seniority | technology | code_challenge_score | technical_interview
('Bernadette', 'Langworth', 'leonard91@yahoo.com', 'Norway', datetime.date(2021, 2, 26), 2, 'Intern', 'Data Engineer', Decimal('3.00'), '3')
('Camryn', 'Reynolds', 'zelda56@hotmail.com', 'Panama', datetime.date(2021, 9, 9), 10, 'Intern', 'Data Engineer', Decimal('2.00'), '10')
('Larue', 'Spinka', 'okey_schultz41@gmail.com', 'Belarus', datetime.date(2020, 4, 14), 4, 'Mid-Level', 'Client Success', Decimal('10.00'), '9')
